# Improved Nonlinear Optimization for Willow Tree Sampling

This notebook solves the nonlinear optimization problem for finding the optimal representative values `z` for a discretized normal distribution, a key step in constructing an implied willow tree. 

The problem is formulated as a nonlinear program with equality and box constraints. Instead of using a black-box penalty method, this notebook uses `JuMP.jl` to explicitly define the objective function and constraints. This approach is more robust and mathematically precise.

The `Ipopt` solver, a state-of-the-art interior-point optimizer for large-scale nonlinear optimization, is used to find the solution.

### Dependencies

Before running the code, make sure you have the necessary Julia packages installed. You can install them by running the following commands in a Julia REPL or a code cell in this notebook:


In [1]:
using JuMP, Ipopt

println("Done loading packages.")

rand_val = rand() * 0.1
println("Random number between 0 and 0.1: ", rand_val)

# --- Problem Data ---
m = 100
q = [0.01558822190112092, 0.0216736949157998, 0.025263154654762213, 0.02794640775787956, 0.03013487069165903, 0.03200474600938565, 0.033649575134551336, 0.03512561663999733, 0.03646962148539734, 0.037707058932142296, 0.03885638269578825, 0.03993143616656965, 0.04094289824450984, 0.04189919785855301, 0.04280711691188369, 0.04280711691188369, 0.04189919785855301, 0.04094289824450984, 0.03993143616656965, 0.03885638269578825, 0.037707058932142296, 0.03646962148539734, 0.03512561663999733, 0.033649575134551336, 0.03200474600938565, 0.03013487069165903, 0.02794640775787956, 0.025263154654762213, 0.0216736949157998, 0.01558822190112092]
Z_bounds = [-3.0, -2.154813341554466, -1.7833838913613136, -1.5339167165110092, -1.3378573012010246, -1.1719610363878912, -1.0252988635663451, -0.8917604546545208, -0.7675197160795298, -0.6499696621466611, -0.5372051182121156, -0.42774236591438064, -0.320351913092292, -0.21394830480998098, -0.10750826831304774, 0.0, 0.10750826831304774, 0.21394830480998098, 0.320351913092292, 0.42774236591438064, 0.5372051182121156, 0.6499696621466611, 0.7675197160795298, 0.8917604546545208, 1.0252988635663451, 1.1719610363878912, 1.3378573012010246, 1.5339167165110092, 1.7833838913613136, 2.154813341554466, 3.0]
z_initial = [-2.5774066707772327 - rand_val, -1.9690986164578899, -1.6586503039361613, -1.435887008856017, -1.254909168794458, -1.0986299499771182, -0.958529659110433, -0.8296400853670254, -0.7087446891130955, -0.5935873901793883, -0.4824737420632481, -0.37404713950333635, -0.2671501089511365, -0.16072828656151436, -0.05375413415652387, 0.05375413415652387, 0.16072828656151436, 0.2671501089511365, 0.37404713950333635, 0.4824737420632481, 0.5935873901793883, 0.7087446891130955, 0.8296400853670254, 0.958529659110433, 1.0986299499771182, 1.254909168794458, 1.435887008856017, 1.6586503039361613, 1.9690986164578899, 2.5774066707772327 + rand_val]

# Define the lower and upper bounds for z
lb = Z_bounds[1:m]
ub = Z_bounds[2:m+1]

# --- JuMP Model ---
# Create a new JuMP model and specify the Ipopt solver
model = Model(Ipopt.Optimizer)

# Set strict tolerance parameters for Ipopt
tol = 1e-14
set_optimizer_attribute(model, "tol", tol)  # Default is 1e-8
set_optimizer_attribute(model, "constr_viol_tol", tol)  # Constraint violation tolerance
set_optimizer_attribute(model, "acceptable_tol", tol / 100)  # Acceptable tolerance
set_optimizer_attribute(model, "dual_inf_tol", tol)  # Dual infeasibility tolerance
set_optimizer_attribute(model, "compl_inf_tol", tol)  # Complementarity tolerance
set_optimizer_attribute(model, "max_iter", 1000)  # Increase max iterations if needed

# Define the variables 'z' with their bounds and initial values
@variable(model, lb[i] <= z[i=1:m] <= ub[i], start = z_initial[i])

# Define the objective function
# min (sum(q_i * z_i^4) - 3)^2
@objective(model, Min, (sum(q[i] * z[i]^4 for i in 1:m) - 3.0)^2)

# Define the equality constraints
# sum(q_i * z_i) = 0
@constraint(model, mean_constraint, sum(q[i] * z[i] for i in 1:m) == 0)

# sum(q_i * z_i^2) = 1
@constraint(model, variance_constraint, sum(q[i] * z[i]^2 for i in 1:m) == 1)


println("Solving the optimization problem with JuMP and Ipopt...")
# Solve the model
optimize!(model)


# --- Results ---
println("\n--- Results ---")
println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))
println("Objective value: ", objective_value(model))

if termination_status(model) == MOI.LOCALLY_SOLVED
    solution_z = value.(z)
    println("\nOptimal z values:")
    println(solution_z)

    # You can also verify the constraints with the solution
    mean_val = sum(q .* solution_z)
    variance_val = sum(q .* solution_z.^2)
    kurtosis_val = sum(q .* solution_z.^4)
    println("\nVerification of constraints:")
    println("Mean: ", mean_val)
    println("Variance: ", variance_val)
    println("Sum of q*z^4 (related to kurtosis): ", kurtosis_val)
else
    println("\nCould not find an optimal solution.")
end

Done loading packages.
Random number between 0 and 0.1: 0.005828290383190705


BoundsError: BoundsError: attempt to access 31-element Vector{Float64} at index [1:100]

### Explanation of the Results

The output of the solver provides several key pieces of information:

- **Termination status:** This tells you how the optimization process ended. `LOCALLY_SOLVED` indicates that the solver found a point that satisfies the first-order optimality conditions, meaning it is a local minimum. For non-convex problems like this one, a local optimum is often the best we can guarantee.
- **Primal status:** This indicates whether a feasible solution was found. `FEASIBLE_POINT` means the solution satisfies all the defined constraints.
- **Objective value:** This is the value of the objective function at the optimal solution. In this case, it is the squared error of the fourth moment from the target value of 3. A value very close to zero indicates a good fit.
- **Optimal z values:** These are the final values of the decision variables that minimize the objective function while satisfying the constraints.
- **Verification of constraints:** This section explicitly calculates the mean, variance, and fourth moment using the optimal `z` values to confirm that the constraints are indeed satisfied to a high degree of precision.